In [1]:
import csv
import osmium
import time
from shapely.geometry import LineString
import geopandas as gpd

without geoms

In [2]:
# import osmium
# import time
# from shapely.geometry import LineString
# import geopandas as gpd

# class WaterwayGeometryExtractor(osmium.SimpleHandler):
#     def __init__(self):
#         super().__init__()
#         self.waterways = []
#         self.count = 0
#         self.error_count = 0
#         self.start_time = time.time()
        
#     def way(self, w):
#         if 'waterway' in w.tags:
#             self.count += 1
            
#             # Show progress
#             if self.count % 1000 == 0:
#                 elapsed = time.time() - self.start_time
#                 print(f"Processed {self.count} waterways in {elapsed:.2f} seconds")
            
#             # Extract coordinates from the way's nodes
#             try:
#                 valid_coords = []
#                 for n in w.nodes:
#                     try:
#                         if n.location.valid():
#                             valid_coords.append((n.lon, n.lat))
#                     except:
#                         continue
                        
#                 if len(valid_coords) >= 2:  # Need at least 2 points for a LineString
#                     self.waterways.append({
#                         'id': w.id,
#                         'waterway_type': w.tags.get('waterway'),
#                         'name': w.tags.get('name', ''),
#                         'geometry': LineString(valid_coords)
#                     })
#                 else:
#                     self.error_count += 1
#             except Exception as e:
#                 self.error_count += 1
#                 if self.error_count % 1000 == 0:
#                     print(f"Error count: {self.error_count}")

# print("Starting waterway geometry extraction...")
# fp = r"C:\Users\Arnell\Downloads\argentina-latest.osm.pbf"
# handler = WaterwayGeometryExtractor()

# try:
#     handler.apply_file(fp)
#     print(f"Processing complete! Found {len(handler.waterways)} valid waterways")
#     print(f"Errors encountered: {handler.error_count}")
    
#     if len(handler.waterways) > 0:
#         # Convert to GeoDataFrame
#         gdf = gpd.GeoDataFrame(handler.waterways, crs="EPSG:4326")
#         output_file = "argentina_waterways_with_geometry.gpkg"
#         gdf.to_file(output_file, driver="GPKG")
#         print(f"Saved to {output_file}")
#     else:
#         print("No valid waterways to save")
        
# except Exception as e:
#     print(f"Error during processing: {e}")

with geoms

In [3]:


# class WaterwayGeometryExtractor(osmium.SimpleHandler):
#     def __init__(self):
#         super().__init__()
#         self.waterways = []
#         self.count = 0
#         self.error_count = 0
#         self.start_time = time.time()
#         self.debug_samples = []  # Store a few problem examples
        
#     def way(self, w):
#         if 'waterway' in w.tags:
#             self.count += 1
            
#             # Show progress
#             if self.count % 10000 == 0:
#                 elapsed = time.time() - self.start_time
#                 print(f"Processed {self.count} waterways in {elapsed:.2f} seconds")
            
#             # DEBUG: Print details of first few ways
#             if self.count <= 5:
#                 print(f"Way {w.id}: {len(w.nodes)} nodes, first node: {w.nodes[0].ref}")
            
#             try:
#                 # Try different node access approach
#                 coords = []
#                 for n in w.nodes:
#                     try:
#                         # Try direct access first
#                         lon, lat = n.lon, n.lat
#                         coords.append((lon, lat))
#                     except:
#                         # Skip invalid nodes
#                         pass
                        
              
#                 if len(coords) >= 2:
#                     # Create a dictionary with basic properties
#                     way_dict = {
#                         'id': w.id,
#                         'waterway': w.tags.get('waterway',''),
#                         'boat':w.tags.get('boat', ''),
#                         'name': w.tags.get('name', '')
#                     }
                    
#                     # # Add ALL other tags as additional properties
#                     # for key, value in w.tags:
#                     #     if key not in ['waterway', 'name']:  # Skip ones we already added
#                     #         # Use safe field naming to avoid GIS software issues
#                     #         safe_key = key.replace(':', '_')  # Replace colons with underscores
#                     #         way_dict[safe_key] = value
                            
#                     # Add geometry
#                     way_dict['geometry'] = LineString(coords)
                    
#                     # Add the complete dictionary
#                     self.waterways.append(way_dict)

#                 else:
#                     if len(self.debug_samples) < 3:
#                         self.debug_samples.append(w.id)
#                     self.error_count += 1
#             except Exception as e:
#                 self.error_count += 1

# print("Starting waterway geometry extraction with modified approach...")
# # fp = r"C:\Users\Arnell\Downloads\argentina-latest.osm.pbf"
# fp = r"C:\Users\Arnell\Downloads\liechtenstein-latest.osm.pbf"
# # fp = r"C:\Users\Arnell\Downloads\asia-latest.osm.pbf"

# # IMPORTANT: Create a LocationHandler to ensure nodes have coordinates
# osm = osmium.io.Reader(fp)
# idx = osmium.index.create_map("sparse_mem_array")
# lh = osmium.NodeLocationsForWays(idx)
# handler = WaterwayGeometryExtractor()

# try:
#     # Use location handler to ensure node coordinates are available
#     osmium.apply(osm, lh, handler)
#     print(f"Processing complete! Found {len(handler.waterways)} valid waterways")
#     print(f"Errors encountered: {handler.error_count}")
#     print(f"Debug samples (problem way IDs): {handler.debug_samples}")
    
#     if len(handler.waterways) > 0:
#         # Convert to GeoDataFrame
#         gdf = gpd.GeoDataFrame(handler.waterways, crs="EPSG:4326")
#         output_file = "liechtenstein_waterways.gpkg"
#         gdf.to_file(output_file, driver="GPKG")
#         print(f"Saved to {output_file}")
#     else:
#         print("No valid waterways to save")
        
# except Exception as e:
#     print(f"Error during processing: {e}")

highways

to geopackage (seems to work)

In [5]:
            
class HighwayGeometryExtractor(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.highways = []
        self.count = 0
        self.error_count = 0
        self.start_time = time.time()
        self.debug_samples = []  # Store a few problem examples
        
    def way(self, w):
        if 'highway' in w.tags:  # Changed from 'waterway' to 'highway'
            self.count += 1
            
            # Show progress
            if self.count % 10000 == 0:
                elapsed = time.time() - self.start_time
                print(f"Processed {self.count} highways in {elapsed:.2f} seconds")
            
            # DEBUG: Print details of first few ways
            if self.count <= 5:
                print(f"Way {w.id}: {len(w.nodes)} nodes, first node: {w.nodes[0].ref}")
            
            try:
                # Try different node access approach
                coords = []
                for n in w.nodes:
                    try:
                        # Try direct access first
                        lon, lat = n.lon, n.lat
                        coords.append((lon, lat))
                    except:
                        # Skip invalid nodes
                        pass
                        
              
                if len(coords) >= 2:
                    # Create a dictionary with basic properties
                    way_dict = {
                        'id': w.id,
                        'highway': w.tags.get('highway',''),  # Changed from 'waterway' to 'highway'
                        'surface': w.tags.get('surface', ''),  # Changed from 'boat' to 'surface' (common highway tag)
                        'name': w.tags.get('name', '')
                    }
                    
                    # # Add ALL other tags as additional properties
                    # for key, value in w.tags:
                    #     if key not in ['highway', 'name']:  # Changed from 'waterway' to 'highway'
                    #         # Use safe field naming to avoid GIS software issues
                    #         safe_key = key.replace(':', '_')  # Replace colons with underscores
                    #         way_dict[safe_key] = value
                            
                    # Add geometry
                    way_dict['geometry'] = LineString(coords)
                    
                    # Add the complete dictionary
                    self.highways.append(way_dict)  # Changed from 'waterways' to 'highways'

                else:
                    if len(self.debug_samples) < 3:
                        self.debug_samples.append(w.id)
                    self.error_count += 1
            except Exception as e:
                self.error_count += 1

print("Starting highway geometry extraction with modified approach...")
# fp = r"C:\Users\Arnell\Downloads\africa-latest.osm.pbf"
# fp = r"C:\Users\Arnell\Downloads\liechtenstein-latest.osm.pbf"
fp = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\europe-latest.osm.pbf"
# fp = r"C:\Users\Arnell\Downloads\asia-latest.osm.pbf"

# IMPORTANT: Create a LocationHandler to ensure nodes have coordinates
osm = osmium.io.Reader(fp)
idx = osmium.index.create_map("sparse_mem_array")
lh = osmium.NodeLocationsForWays(idx)
handler = HighwayGeometryExtractor()  # Changed class name

try:
    # Use location handler to ensure node coordinates are available
    osmium.apply(osm, lh, handler)
    print(f"Processing complete! Found {len(handler.highways)} valid highways")  # Changed from 'waterways' to 'highways'
    print(f"Errors encountered: {handler.error_count}")
    print(f"Debug samples (problem way IDs): {handler.debug_samples}")
    
    if len(handler.highways) > 0:  # Changed from 'waterways' to 'highways'
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(handler.highways, crs="EPSG:4326")  # Changed from 'waterways' to 'highways'
        # output_file = "africa_highways.gpkg"  # Changed filename
        output_file = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\osm\osm_regional_250521\europe_highways.gpkg"  # Changed filename
        gdf.to_file(output_file, driver="GPKG")
        print(f"Saved to {output_file}")
    else:
        print("No valid highways to save")  # Changed from 'waterways' to 'highways'
        
except Exception as e:
    print(f"Error during processing: {e}")

Starting highway geometry extraction with modified approach...
Error during processing: bad allocation


testing error handling for highways

In [ ]:
import osmium
import time
import os
import gc
import psutil
import sys
from shapely.geometry import LineString
import geopandas as gpd

class ChunkedHighwayExtractor(osmium.SimpleHandler):
    def __init__(self, max_highways=5000, max_memory_percent=70):
        super().__init__()
        self.highways = []
        self.count = 0
        self.total_count = 0
        self.error_count = 0
        self.start_time = time.time()
        self.debug_samples = []
        self.max_highways = max_highways
        self.chunk_ready = False
        self.max_memory_percent = max_memory_percent
        
    def way(self, w):
        # Check memory usage and stop if we're using too much
        if psutil.virtual_memory().percent > self.max_memory_percent:
            print(f"WARNING: Memory usage at {psutil.virtual_memory().percent}% - stopping to save data")
            self.chunk_ready = True
            return True
            
        if 'highway' in w.tags:
            self.count += 1
            self.total_count += 1
            
            # Show progress
            if self.total_count % 10000 == 0:
                elapsed = time.time() - self.start_time
                mem = psutil.virtual_memory().percent
                print(f"Processed {self.total_count} highways in {elapsed:.2f} seconds (Memory: {mem}%)")
            
            try:
                # Only collect coordinates if we need them (memory optimization)
                if len(self.highways) < self.max_highways:
                    coords = []
                    for n in w.nodes:
                        try:
                            lon, lat = n.lon, n.lat
                            coords.append((lon, lat))
                        except:
                            pass
                            
                    if len(coords) >= 2:
                        # Minimal attributes to save memory
                        way_dict = {
                            'osm_id': w.id,
                            'highway': w.tags.get('highway',''),
                            'geometry': LineString(coords)
                        }
                        
                        self.highways.append(way_dict)

                # Check if we've reached capacity
                if len(self.highways) >= self.max_highways:
                    self.chunk_ready = True
                    return True  # Signal to stop processing
            except Exception as e:
                self.error_count += 1
                
        return True
    
    def reset(self):
        """Reset the highways list to free memory"""
        self.highways = []
        self.count = 0
        self.chunk_ready = False
        # Force garbage collection
        gc.collect()


def process_osm_in_chunks(input_file, output_dir, chunk_size=5000, output_prefix="highways_chunk"):
    """Process OSM file in chunks to manage memory usage"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Starting chunked highway extraction from {input_file}")
    print(f"Saving chunks of {chunk_size} highways each to {output_dir}")
    
    chunk_num = 1
    total_saved = 0
    
    # Setup OSM reader and handler
    handler = ChunkedHighwayExtractor(max_highways=chunk_size)
    
    try:
        while True:
            print(f"\nProcessing chunk {chunk_num}...")
            
            # Open the OSM file with limited cache to reduce memory usage
            reader = osmium.io.Reader(input_file)
            
            # Use standard index type that's supported in all installations
            idx = osmium.index.create_map("sparse_mem_array")
            location_handler = osmium.NodeLocationsForWays(idx)
            
            try:
                # Apply the handler
                osmium.apply(reader, location_handler, handler)
                print(f"Chunk {chunk_num} processing complete!")
                
                # Save the chunk if we have data
                if len(handler.highways) > 0:
                    save_chunk(handler.highways, output_dir, f"{output_prefix}_{chunk_num}.gpkg")
                    total_saved += len(handler.highways)
                    print(f"Saved {len(handler.highways)} highways to chunk {chunk_num}")
                    print(f"Total highways saved so far: {total_saved}")
                else:
                    print("No more highways to process")
                    break
                
                # Reset for next chunk
                handler.reset()
                chunk_num += 1
                
                # Explicitly close and clean up resources
                del location_handler
                del idx
                reader.close()
                
                # Force aggressive garbage collection
                gc.collect()
                
                # Break if handler didn't indicate there's more to process
                if not handler.chunk_ready:
                    break
                    
            except MemoryError as me:
                print(f"Memory error processing chunk {chunk_num}: {me}")
                
                # Emergency save of current data
                if len(handler.highways) > 0:
                    try:
                        save_chunk(handler.highways, output_dir, f"{output_prefix}_{chunk_num}_partial_emergency.gpkg")
                        total_saved += len(handler.highways)
                        print(f"Emergency save of partial chunk with {len(handler.highways)} highways")
                    except Exception as save_error:
                        print(f"Failed to save partial chunk: {save_error}")
                
                # Free as much memory as possible
                del location_handler
                del idx
                reader.close()
                handler.reset()
                gc.collect()
                
                # Try with a smaller chunk size
                chunk_size = max(1000, int(chunk_size * 0.5))
                handler.max_highways = chunk_size
                print(f"Reduced chunk size to {chunk_size} for next attempt")
                chunk_num += 1
                
            except Exception as e:
                print(f"Error processing chunk {chunk_num}: {e}")
                
                # Try to save what we have so far
                if len(handler.highways) > 0:
                    try:
                        save_chunk(handler.highways, output_dir, f"{output_prefix}_{chunk_num}_partial.gpkg")
                        total_saved += len(handler.highways)
                        print(f"Saved partial chunk with {len(handler.highways)} highways")
                    except Exception as save_error:
                        print(f"Failed to save partial chunk: {save_error}")
                
                # Clean up before next attempt
                try:
                    reader.close()
                except:
                    pass
                    
                handler.reset()
                chunk_num += 1
                gc.collect()
                
            finally:
                # Make sure reader is closed
                try:
                    reader.close()
                except:
                    pass
    
    finally:
        # Final cleanup
        print(f"\nProcessing complete! Saved {total_saved} highways across {chunk_num-1} chunks")
        print(f"Errors encountered: {handler.error_count}")
        print(f"Debug samples (problem way IDs): {handler.debug_samples}")


def save_chunk(highways, output_dir, filename):
    """Save a chunk of highways to a geopackage file"""
    
    output_path = os.path.join(output_dir, filename)
    
    # Sanitize field names
    for item in highways:
        if 'FID' in item:
            item['fid_value'] = item.pop('FID')
    
    try:
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(highways, geometry='geometry', crs="EPSG:4326")
        
        # Save to file - try with smaller chunks if needed
        try:
            gdf.to_file(output_path, driver="GPKG")
        except Exception as e:
            print(f"Error saving complete chunk: {e}, trying with smaller parts...")
            
            # Try saving in even smaller chunks if needed
            rows = len(gdf)
            subchunk_size = 1000  # Reduced from 5000 to 1000
            for i in range(0, rows, subchunk_size):
                end_idx = min(i + subchunk_size, rows)
                subchunk_path = f"{output_path.replace('.gpkg', '')}_part_{i}_{end_idx}.gpkg"
                gdf.iloc[i:end_idx].to_file(subchunk_path, driver="GPKG")
                print(f"Saved subchunk {i}-{end_idx} to {subchunk_path}")
                
        print(f"Saved to {output_path}")
        
    except Exception as e:
        print(f"Critical error in save_chunk: {e}")
        # Last resort - try CSV format without geometry
        try:
            csv_path = output_path.replace('.gpkg', '.csv')
            # Extract just the attributes without geometry
            highway_attrs = []
            for h in highways:
                attrs = {k: v for k, v in h.items() if k != 'geometry'}
                highway_attrs.append(attrs)
                
            import pandas as pd
            pd.DataFrame(highway_attrs).to_csv(csv_path, index=False)
            print(f"Saved attributes only (no geometry) to {csv_path}")
        except:
            print("All save attempts failed")


if __name__ == "__main__":
    input_file = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\europe-latest.osm.pbf"
    output_dir = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\osm\osm_regional_250521\europe_chunks"
    
    # Process in smaller chunks to manage memory better
    process_osm_in_chunks(input_file, output_dir, chunk_size=5000, output_prefix="europe_highways")

Starting chunked highway extraction from C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\europe-latest.osm.pbf
Saving chunks of 5000 highways each to C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\osm\osm_regional_250521\europe_chunks

Processing chunk 1...
Memory error processing chunk 1: bad allocation
Reduced chunk size to 2500 for next attempt

Processing chunk 2...
Memory error processing chunk 2: bad allocation
Reduced chunk size to 1250 for next attempt

Processing chunk 3...
Memory error processing chunk 3: bad allocation
Reduced chunk size to 1000 for next attempt

Processing chunk 4...
Memory error processing chunk 4: bad allocation
Reduced chunk size to 1000 for next attempt

Processing chunk 5...
Memory error processing chunk 5: bad allocation
Reduced chunk size to 1000 for next attempt

Processing chunk 6...

In [ ]:
            
# class HighwayGeometryExtractor(osmium.SimpleHandler):
#     def __init__(self):
#         super().__init__()
#         self.highways = []
#         self.count = 0
#         self.error_count = 0
#         self.start_time = time.time()
#         self.debug_samples = []  # Store a few problem examples
        
#     def way(self, w):
#         if 'highway' in w.tags:  # Changed from 'waterway' to 'highway'
#             self.count += 1
            
#             # Show progress
#             if self.count % 10000 == 0:
#                 elapsed = time.time() - self.start_time
#                 print(f"Processed {self.count} highways in {elapsed:.2f} seconds")
            
#             # DEBUG: Print details of first few ways
#             if self.count <= 5:
#                 print(f"Way {w.id}: {len(w.nodes)} nodes, first node: {w.nodes[0].ref}")
            
#             try:
#                 # Try different node access approach
#                 coords = []
#                 for n in w.nodes:
#                     try:
#                         # Try direct access first
#                         lon, lat = n.lon, n.lat
#                         coords.append((lon, lat))
#                     except:
#                         # Skip invalid nodes
#                         pass
                        
              
#                 if len(coords) >= 2:
#                     # Create a dictionary with basic properties
#                     way_dict = {
#                         'id': w.id,
#                         'highway': w.tags.get('highway',''),  # Changed from 'waterway' to 'highway'
#                         'surface': w.tags.get('surface', ''),  # Changed from 'boat' to 'surface' (common highway tag)
#                         'name': w.tags.get('name', '')
#                     }
                    
#                     # # Add ALL other tags as additional properties
#                     # for key, value in w.tags:
#                     #     if key not in ['highway', 'name']:  # Changed from 'waterway' to 'highway'
#                     #         # Use safe field naming to avoid GIS software issues
#                     #         safe_key = key.replace(':', '_')  # Replace colons with underscores
#                     #         way_dict[safe_key] = value
                            
#                     # Add geometry
#                     way_dict['geometry'] = LineString(coords)
                    
#                     # Add the complete dictionary
#                     self.highways.append(way_dict)  # Changed from 'waterways' to 'highways'

#                 else:
#                     if len(self.debug_samples) < 3:
#                         self.debug_samples.append(w.id)
#                     self.error_count += 1
#             except Exception as e:
#                 self.error_count += 1

# print("Starting highway geometry extraction with modified approach...")
# # fp = r"C:\Users\Arnell\Downloads\africa-latest.osm.pbf"
# # fp = r"C:\Users\Arnell\Downloads\liechtenstein-latest.osm.pbf"
# # fp = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\australia-oceania-latest.osm.pbf"
# fp = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\europe-latest.osm.pbf"

# # fp = r"C:\Users\Arnell\Downloads\asia-latest.osm.pbf"

# # IMPORTANT: Create a LocationHandler to ensure nodes have coordinates
# osm = osmium.io.Reader(fp)
# idx = osmium.index.create_map("sparse_mem_array")
# lh = osmium.NodeLocationsForWays(idx)
# handler = HighwayGeometryExtractor()  # Changed class name

# try:
#     # Use location handler to ensure node coordinates are available
#     osmium.apply(osm, lh, handler)
#     print(f"Processing complete! Found {len(handler.highways)} valid highways")  # Changed from 'waterways' to 'highways'
#     print(f"Errors encountered: {handler.error_count}")
#     print(f"Debug samples (problem way IDs): {handler.debug_samples}")
    
#     if len(handler.highways) > 0:  # Changed from 'waterways' to 'highways'
#         # Convert to GeoDataFrame
#         gdf = gpd.GeoDataFrame(handler.highways, crs="EPSG:4326")  # Changed from 'waterways' to 'highways'
#         # output_file = "africa_highways.gpkg"  # Changed filename
#         output_file = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\work_in_progress\roads\osm\osm_regional_250521\australia_oceania_highways.gpkg"  # Changed filename
#         gdf.to_file(output_file, driver="GPKG")
#         print(f"Saved to {output_file}")
#     else:
#         print("No valid highways to save")  # Changed from 'waterways' to 'highways'
        
# except Exception as e:
#     print(f"Error during processing: {e}")

Starting highway geometry extraction with modified approach...
Way 2850: 3 nodes, first node: 691734
Way 2857: 8 nodes, first node: 441520023
Way 1881386: 5 nodes, first node: 1805891935
Way 1881388: 2 nodes, first node: 17396811
Way 1881389: 6 nodes, first node: 1805891863
Processed 10000 highways in 77.12 seconds
Processed 20000 highways in 77.93 seconds
Processed 30000 highways in 78.81 seconds
Processed 40000 highways in 79.64 seconds
Processed 50000 highways in 80.52 seconds
Processed 60000 highways in 81.48 seconds
Processed 70000 highways in 82.37 seconds
Processed 80000 highways in 83.45 seconds
Processed 90000 highways in 84.34 seconds
Processed 100000 highways in 85.27 seconds
Processed 110000 highways in 86.06 seconds
Processed 120000 highways in 86.88 seconds
Processed 130000 highways in 87.78 seconds
Processed 140000 highways in 88.55 seconds
Processed 150000 highways in 89.44 seconds
Processed 160000 highways in 90.35 seconds
Processed 170000 highways in 92.93 seconds
Pro

In [ ]:
# class HighwayGeometryExtractor(osmium.SimpleHandler):
#     def __init__(self):
#         super().__init__()
#         self.highways = []
#         self.count = 0
#         self.error_count = 0
#         self.start_time = time.time()
#         self.debug_samples = []  # Store a few problem examples
        
#     def way(self, w):
#         if 'highway' in w.tags:  # Changed from 'waterway' to 'highway'
#             self.count += 1
            
#             # Show progress
#             if self.count % 10000 == 0:
#                 elapsed = time.time() - self.start_time
#                 print(f"Processed {self.count} highways in {elapsed:.2f} seconds")
            
#             # DEBUG: Print details of first few ways
#             if self.count <= 5:
#                 print(f"Way {w.id}: {len(w.nodes)} nodes, first node: {w.nodes[0].ref}")
            
#             try:
#                 # Try different node access approach
#                 coords = []
#                 for n in w.nodes:
#                     try:
#                         # Try direct access first
#                         lon, lat = n.lon, n.lat
#                         coords.append((lon, lat))
#                     except:
#                         # Skip invalid nodes
#                         pass
                        
              
#                 if len(coords) >= 2:
#                     # Create a dictionary with basic properties
#                     way_dict = {
#                         'osm_id': w.id,  # Changed from 'id' to 'osm_id' to avoid conflicts
#                         'highway': w.tags.get('highway',''),
#                         'surface': w.tags.get('surface', ''),  # Changed from 'boat' to 'surface' (common highway tag)
#                         'name': w.tags.get('name', '')
#                     }
                    
#                     # # Add ALL other tags as additional properties
#                     # for key, value in w.tags:
#                     #     if key not in ['highway', 'name']:
#                     #         # Use safe field naming to avoid GIS software issues
#                     #         safe_key = key.replace(':', '_')  # Replace colons with underscores
                            
#                     #         # Avoid reserved field names
#                     #         if safe_key.upper() in ['FID', 'OGC_FID']:
#                     #             safe_key = f"{safe_key}_value"
                                
#                     #         way_dict[safe_key] = value
                            
#                     # Add geometry
#                     way_dict['geometry'] = LineString(coords)
                    
#                     # Add the complete dictionary
#                     self.highways.append(way_dict)

#                 else:
#                     if len(self.debug_samples) < 3:
#                         self.debug_samples.append(w.id)
#                     self.error_count += 1
#             except Exception as e:
#                 self.error_count += 1

# print("Starting highway geometry extraction with modified approach...")
# # fp = r"C:\Users\Arnell\Downloads\europe-latest.osm.pbf"
# fp = r"C:\Users\Arnell\OneDrive - Food and Agriculture Organization\project_work\p0002_primary_forest_support\raw\roads\osm\osm_regional_250521\europe-latest.osm.pbf"

# # fp = r"C:\Users\Arnell\Downloads\liechtenstein-latest.osm.pbf"
# # fp = r"C:\Users\Arnell\Downloads\asia-latest.osm.pbf"

# # IMPORTANT: Create a LocationHandler to ensure nodes have coordinates
# osm = osmium.io.Reader(fp)
# idx = osmium.index.create_map("sparse_mem_array")
# lh = osmium.NodeLocationsForWays(idx)
# handler = HighwayGeometryExtractor()  # Changed class name

# # try:
# #     # Use location handler to ensure node coordinates are available
# #     osmium.apply(osm, lh, handler)
# #     print(f"Processing complete! Found {len(handler.highways)} valid highways")
# #     print(f"Errors encountered: {handler.error_count}")
# #     print(f"Debug samples (problem way IDs): {handler.debug_samples}")
    
# #     if len(handler.highways) > 0:
# #         # Convert to GeoDataFrame with explicit geometry column
# #         gdf = gpd.GeoDataFrame(handler.highways, geometry='geometry', crs="EPSG:4326")
        
# #         # Try saving with error handling
# #         try:
# #             output_file = "argentina_highways.gpkg"
# #             gdf.to_file(output_file, driver="GPKG")
# #             print(f"Successfully saved to {output_file}")
# #         except Exception as save_error:
# #             print(f"Error with GPKG format: {save_error}")
# #             # Try alternate format as fallback
# #             output_file = "argentina_highways.geojson"
# #             gdf.to_file(output_file, driver="GeoJSON")
# #             print(f"Saved as GeoJSON instead: {output_file}")
# #     else:
# #         print("No valid highways to save")
        
# # except Exception as e:
# #     print(f"Error during processing: {e}")
# try:
#     # Use location handler to ensure node coordinates are available
#     osmium.apply(osm, lh, handler)
#     print(f"Processing complete! Found {len(handler.highways)} valid highways")
#     print(f"Errors encountered: {handler.error_count}")
#     print(f"Debug samples (problem way IDs): {handler.debug_samples}")
    
#     if len(handler.highways) > 0:
#         # First, sanitize field names to prevent errors
#         for item in handler.highways:
#             # Rename problematic fields if any
#             if 'FID' in item:
#                 item['fid_value'] = item.pop('FID')
        
#         # Convert to GeoDataFrame with explicit geometry column
#         gdf = gpd.GeoDataFrame(handler.highways, geometry='geometry', crs="EPSG:4326")
        
#         # Attempt multiple save formats with error handling
#         try:
#             # Try saving as GeoPackage
#             output_file = "argentina_highways.gpkg"
#             gdf.to_file(output_file, driver="GPKG")
#             print(f"Successfully saved to {output_file}")
#         except Exception as save_error:
#             print(f"Error saving GeoPackage: {save_error}")
            
#             # Try saving in chunks
#             print("Attempting to save in chunks...")
#             chunk_size = 100000  # Adjust based on your system capabilities
#             total_chunks = (len(gdf) + chunk_size - 1) // chunk_size
            
#             for i in range(total_chunks):
#                 start_idx = i * chunk_size
#                 end_idx = min((i + 1) * chunk_size, len(gdf))
#                 chunk = gdf.iloc[start_idx:end_idx]
                
#                 try:
#                     chunk_file = f"europe_highways_chunk_{i+1}_of_{total_chunks}.gpkg"
#                     chunk.to_file(chunk_file, driver="GPKG")
#                     print(f"Saved chunk {i+1}/{total_chunks} to {chunk_file}")
#                 except Exception as chunk_error:
#                     print(f"Failed to save chunk {i+1}: {chunk_error}")
                    
#                     # Last resort - try GeoJSON format for this chunk
#                     try:
#                         geojson_file = f"europe_highways_chunk_{i+1}.geojson"
#                         chunk.to_file(geojson_file, driver="GeoJSON")
#                         print(f"Saved chunk {i+1} as GeoJSON to {geojson_file}")
#                     except Exception as gj_error:
#                         print(f"All save attempts failed for chunk {i+1}: {gj_error}")
#     else:
#         print("No valid highways to save")
        
# except Exception as e:
#     print(f"Error during processing: {e}")

Starting highway geometry extraction with modified approach...
Error during processing: bad allocation


error handling better

In [5]:
class HighwayGeometryExtractor(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.highways = []
        self.count = 0
        self.error_count = 0
        self.start_time = time.time()
        self.debug_samples = []  # Store a few problem examples
        
    def way(self, w):
        if 'highway' in w.tags:  # Changed from 'waterway' to 'highway'
            self.count += 1
            
            # Show progress
            if self.count % 10000 == 0:
                elapsed = time.time() - self.start_time
                print(f"Processed {self.count} highways in {elapsed:.2f} seconds")
            
            # DEBUG: Print details of first few ways
            if self.count <= 5:
                print(f"Way {w.id}: {len(w.nodes)} nodes, first node: {w.nodes[0].ref}")
            
            try:
                # Try different node access approach
                coords = []
                for n in w.nodes:
                    try:
                        # Try direct access first
                        lon, lat = n.lon, n.lat
                        coords.append((lon, lat))
                    except:
                        # Skip invalid nodes
                        pass
                        
              
                if len(coords) >= 2:
                    # Create a dictionary with basic properties
                    way_dict = {
                        'osm_id': w.id,
                        'highway': w.tags.get('highway',''),
                        'name': w.tags.get('name', '')
                    }
                    
                    # # Add ALL other tags as additional properties
                    # for key, value in w.tags:
                    #     if key not in ['highway', 'name']:
                    #         # Use safe field naming to avoid GIS software issues
                    #         safe_key = key.replace(':', '_')
                            
                    #         # Handle case-sensitive duplicates like fixme/FIXME
                    #         if safe_key.upper() in ['FID', 'OGC_FID']:
                    #             safe_key = f"{safe_key}_value"
                    #         # Handle case sensitivity issues specifically
                    #         elif safe_key.upper() == "FIXME" and safe_key != "FIXME":
                    #             safe_key = "fixme_detail"
                    #         elif safe_key == "FIXME":
                    #             safe_key = "FIXME_detail"
                                
                    #         way_dict[safe_key] = value
                            
                    # Add geometry
                    way_dict['geometry'] = LineString(coords)
                    
                    # Add the complete dictionary
                    self.highways.append(way_dict)

                else:
                    if len(self.debug_samples) < 3:
                        self.debug_samples.append(w.id)
                    self.error_count += 1
            except Exception as e:
                self.error_count += 1

# Initialize OSM reader and handler
print("Starting highway geometry extraction with modified approach...")
fp = r"C:\Users\Arnell\Downloads\argentina-latest.osm.pbf"
osm = osmium.io.Reader(fp)
idx = osmium.index.create_map("sparse_mem_array")
lh = osmium.NodeLocationsForWays(idx)
handler = HighwayGeometryExtractor()

# At save time:
try:
    # Use location handler to ensure node coordinates are available
    osmium.apply(osm, lh, handler)
    print(f"Processing complete! Found {len(handler.highways)} valid highways")
    print(f"Errors encountered: {handler.error_count}")
    
    if len(handler.highways) > 0:
        # Rename any problematic field names - particularly looking for duplicates
        field_names_lower = set()
        for item in handler.highways:
            keys_to_rename = []
            for key in item:
                if key.lower() in field_names_lower:
                    # This is a duplicate when case-insensitive
                    keys_to_rename.append(key)
                else:
                    field_names_lower.add(key.lower())
            
            # Rename any problematic keys
            for key in keys_to_rename:
                item[f'{key}_alt'] = item.pop(key)
        
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(handler.highways, geometry='geometry', crs="EPSG:4326")
        
        # Try saving
        output_file = "argentina_highways.gpkg"
        gdf.to_file(output_file, driver="GPKG")
        print(f"Saved to {output_file}")
    else:
        print("No valid highways to save")
        
except Exception as e:
    print(f"Error during processing: {e}")

Starting highway geometry extraction with modified approach...
Way 4624762: 3 nodes, first node: 247877436
Way 4624877: 6 nodes, first node: 4338404152
Way 4651802: 3 nodes, first node: 29603531
Way 4662390: 11 nodes, first node: 3045993933
Way 4662391: 15 nodes, first node: 333464650
Processed 10000 highways in 8.36 seconds
Processed 20000 highways in 8.84 seconds
Processed 30000 highways in 9.31 seconds
Processed 40000 highways in 9.76 seconds
Processed 50000 highways in 10.17 seconds
Processed 60000 highways in 10.56 seconds
Processed 70000 highways in 11.00 seconds
Processed 80000 highways in 11.45 seconds
Processed 90000 highways in 11.91 seconds
Processed 100000 highways in 12.36 seconds
Processed 110000 highways in 12.84 seconds
Processed 120000 highways in 13.29 seconds
Processed 130000 highways in 13.73 seconds
Processed 140000 highways in 14.19 seconds
Processed 150000 highways in 14.64 seconds
Processed 160000 highways in 15.17 seconds
Processed 170000 highways in 15.68 seco

In [ ]:
gdf.head()